<img src="https://storage.googleapis.com/kaggle-competitions/kaggle/3136/logos/header.png"> 
<p style="text-align:center;">
    <small>Fonte: Kaggle [1]</small>
</p>

## Titanic dataset
 - **Problema**: Dizer se uma pessoa a bordo do Titanic irá morrer ou sobreviver dado um conjunto de características sobre ela.
 - Conhecer os dados, preprocessamento, aplicação do modelo e avaliação do desempenho;

| Característica/feature | Descrição         |
| :------------- |:-------------     | 
| survived       | Sobreviveu ou não | 
| pclass         | Classe social     |
| sex            | Gênero            | 
| age            | Idade em anos     |
| sibsp          | Relação familiar com alguém à bordo (irmão, irmã, esposa) |
| parch          | Relação familiar com alguém à bordo (pais, filhos) |
| ticket         | nº do ticket      |
| fare           | Quanto pagou o ticket |
| cabin          | nº da cabine      |
| embarked       | Cidade em que a pessoa embarcou ( C = Cherbourg, Q = Queenstown, S = Southampton ) |

## Explorar os dados

In [ ]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv("../datasets/titanic/train.csv")

In [ ]:
train

In [ ]:
train.dtypes

In [ ]:
train.Survived.value_counts()

In [ ]:
plt = train.Survived.value_counts().plot(kind="bar")
plt.set_xlabel("Sobreviveu ou não")
plt.set_ylabel("Quantidade de passageiros")

In [ ]:
plt = train.Pclass.value_counts().sort_index().plot(kind='bar', title='')
plt.set_xlabel("Pclass")
plt.set_ylabel("Quantidade de passageiros")

In [ ]:
train.Pclass.value_counts().sort_index()

### Probabilidade de sobrevivência em cada classe socioeconômica

In [ ]:
pclass1 = train.loc[(train['Pclass'] == 1), ['Pclass', 'Survived']]
pclass2 = train.loc[(train['Pclass'] == 2), ['Pclass', 'Survived']]
pclass3 = train.loc[(train['Pclass'] == 3), ['Pclass', 'Survived']]

In [ ]:
prob1 = pclass1["Survived"].mean()
prob2 = pclass2["Survived"].mean()
prob3 = pclass3["Survived"].mean()

In [ ]:
plt = pd.Series([prob1, prob2, prob3], [1,2,3]).plot(kind="bar")
plt.set_xlabel('Classe socioeconômica')
plt.set_ylabel('Probabilidade de sobrevivência')

### Probabilidade de sobrevivência baseada no gênero

In [ ]:
male = train.loc[(train["Sex"] == "male"), ["Sex", "Survived"]]
female = train.loc[(train["Sex"] == "female"), ["Sex", "Survived"]]

In [ ]:
male

In [ ]:
prob_male = male["Survived"].mean()
prob_female = female["Survived"].mean()

In [ ]:
plt = pd.Series([prob_male, prob_female], ["Homens", "Mulheres"]).plot(kind="bar")
plt.set_ylabel("Probabilidade de Sobrevivência")

## Preprocessamento

 - Remover características sem importância;
 - Tratar valores nulos;
 - Normalizar/padronizar;
 - Transformar variáveis categórias (**gênero, cidade que embarcou**) em variáveis númericas;

### Remoção de features pouco importantes

In [ ]:
test = pd.read_csv("../datasets/titanic/test.csv")
train.loc[:, ["Ticket", "PassengerId", "Cabin"]]


In [ ]:
train = train.drop(columns=["Ticket", "PassengerId", "Cabin", "Name"])
test = test.drop(columns=["Ticket", "PassengerId", "Cabin", "Name"])

In [ ]:
test

### Tratando valores nulos

In [ ]:
columns_names = test.columns
for c in columns_names:
    print(c,":", test[c].isnull().sum())

In [ ]:
train

In [ ]:
median_age_train = train["Age"].median()
train.loc[:, ["Age"]] = train["Age"].fillna(median_age_train)

In [ ]:
train = train.dropna(axis=0)

In [ ]:
test

In [ ]:
median_age_test = test["Age"].median()
test.loc[:, ["Age"]] = test["Age"].fillna(median_age_test)

In [ ]:
median_fare_test = test["Fare"].median()
test.loc[:, ["Fare"]] = test["Fare"].fillna(median_fare_test)

In [ ]:
test

### Tratamento das variáveis categóricas
 - One-hot-encoding

In [ ]:
train

In [ ]:
def replace_female(gender):
    if gender == "female":
        return 1
    return 0
train.loc[:, "Sex"].apply(replace_female)
test.loc[:, "Sex"].apply(replace_female)

In [ ]:
female_train = train.loc[:, "Sex"].apply(replace_female)
female_test = test.loc[:, "Sex"].apply(replace_female)

In [ ]:
train.loc[:, "Female"] = female_train
test.loc[:, "Female"] = female_test

In [ ]:
def replace_male(gender):
    if gender == "female":
        return 0
    return 1

In [ ]:
male_train = train.loc[:, "Sex"].apply(replace_male)
male_test = test.loc[:, "Sex"].apply(replace_male)

In [ ]:
train.loc[:, "Male"] = male_train
test.loc[:, "Male"] = male_test

In [ ]:
train = train.drop(columns=["Sex"])
test = test.drop(columns=["Sex"])

In [ ]:
train["Embarked"] = train["Embarked"].map({"C":0, "Q":1, "S":2})
test["Embarked"] = test["Embarked"].map({"C":0, "Q":1, "S":2})

### Padronização

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
#age_train = train["Age"].to_numpy()
age_test = test["Age"].to_numpy()

#fare_train = train["Fare"].to_numpy()
fare_test = test["Fare"].to_numpy()

In [ ]:
#age_train = scaler.fit_transform(age_train.reshape(-1, 1))
age_test = scaler.fit_transform(age_test.reshape(-1, 1))
#fare_train = scaler.fit_transform(fare_train.reshape(-1, 1))
fare_test = scaler.fit_transform(fare_test.reshape(-1, 1))

In [ ]:
#train.loc[:, "Age"] = age_train
test.loc[:, "Age"] = age_test

#train.loc[:, "Fare"] = fare_train
test.loc[:, "Fare"] = fare_test

## Aplicação do Modelo ─ árvore de decisão

In [ ]:
from sklearn import tree

In [ ]:
classifier = tree.DecisionTreeClassifier(random_state)

In [ ]:
X = train.loc[:, "Pclass":"Male"]

In [ ]:
y = train.loc[:, "Survived"]

In [ ]:
classifier = classifier.fit(X, y)

In [ ]:
predictions = classifier.predict(test)

In [ ]:
predictions.shape

## Avaliação de desempenho

In [ ]:
from sklearn.metrics import accuracy_score, plot_confusion_matrix

In [ ]:
ground_truth = pd.read_csv("../datasets/titanic/gender_submission.csv")

In [ ]:
accuracy = accuracy_score(ground_truth["Survived"], predictions)

In [ ]:
print("Acurrácia: {:.2f}%".format(accuracy*100))

In [ ]:
X_test = test
y_test = ground_truth["Survived"]

In [ ]:
y_test

In [ ]:
plot_confusion_matrix(classifier, X_test, y_test)

## Tarefa
 - Alterar os parâmetros `criterion`, `splitter`, `max_depth` da árvore de decisão e verificar se a acurácia do modelo melhora (ver referência **[2]**).

## Referências
[[1]](https://www.kaggle.com/c/titanic/) Titanic - Machine Learning from Disaster

[[2]](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn-tree-decisiontreeclassifier) Documentação do scikitlean para árvores de decisão